In [1]:
from __future__ import print_function
import fitz, sys

def split_pdf(finput):
    doc = fitz.open(finput)
    page = doc.loadPage(0)
    if int(page.MediaBox[2]) > 800.0:
        res = fitz.open() 

        for spage in doc:
            r = spage.rect  
            d = fitz.Rect(spage.CropBoxPosition, spage.CropBoxPosition)  
            
            r1 = fitz.Rect(0, 0, r.width*0.5, r.height)
            r2 = fitz.Rect(r.width*0.5, 0, r.width, r.height)
            rect_list = [r1, r2]
    
            for rx in rect_list:  
                rx += d 
                page = res.newPage(-1, width = rx.width, height = rx.height)
                page.showPDFpage(page.rect, doc, spage.number, clip = rx)

        res.save(doc.name[:-4]+'_sp.pdf', garbage = 3, deflate = True)
        fname = doc.name[:-4]+'_sp.pdf'
        doc = fitz.open(fname)  # open document
        out = open("output.txt", "wb")  # open text output
        for i, page in enumerate(doc):  # iterate the document pages
            if i%2 == 0:
                text = page.getText().encode("utf8")  # get plain text (is in UTF-8)
                out.write(text)  # write text of page
                out.write(bytes((12,)))  # write page delimiter (form feed 0x0C)
        out.close()
        
        with open('output.txt', 'r',-1,'utf-8') as f:
            text=f.read()
        
        return text
    
    else:
        return 0

In [2]:
import pdftotext
import os

def text22text(finput):
    
    fout = os.path.join(os.getcwd(), 'output.txt')
    
    if '.pdf' in finput:
         
        with open(finput, "rb") as f:
            pdf = pdftotext.PDF(f)

        with open('output.txt', 'w',-1,'utf-8') as f:
            f.write("\n\n".join(pdf))
    
        with open('output.txt', 'r',-1,'utf-8') as f:
            text=f.read()
            
    elif '.HWP' or '.hwp' in finput:
        hwp2txt_cmd = r'hwp5txt --output={} {}'.format(fout, finput)
        os.system(hwp2txt_cmd)
        with open(fout,'r',encoding='UTF8') as f:
            text=f.read()
            
    return text

In [3]:
import glob
from PIL import Image
from PyPDF2 import PdfFileReader
from pdf2image import convert_from_bytes
from pathos.multiprocessing import ProcessingPool as Pool

def image22text(finput, seqnum):
    img_name = os.path.join(r'C:\pgm_file', 'img_file', seqnum) 
    if '.hwp' in finput:
        return 0
    
    if not os.path.exists(img_name):
        os.mkdir(img_name)
        images = convert_from_bytes(open(finput, 'rb').read(), dpi=300, output_folder = img_name, fmt="png")
    
    img_list = list()
    for i in range(len(os.listdir(img_name))):
        img_list.append(img_name)
        
    with Pool(10) as p:
        text = p.map(image_multi, img_list, os.listdir(img_name))
    return " ".join(text)

def image_multi(img_name, img):
    import cv2
    import os
    import pytesseract
    img = cv2.imread(os.path.join(img_name, img))
    img = cv2.resize(img, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_CUBIC)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 85, 11)

    custom_config = r'-l kor -c tessedit_char_blacklist=ABCDEFGHIJKLMNOPQRSTUVWXYZ --psm 6'
    temp = pytesseract.image_to_string(thresh, config = custom_config, lang='kor')
    return temp

In [4]:
import os
import pandas as pd 

tlist, ilist = list(), list()
xname = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', f"{'xlsx_#1_file_list'}.xlsx")
df = pd.read_excel(xname, index_col=0)

for i in range(len(df)):
    
    finput = df.loc[i, 'path']
    
    if 'pdf' in finput:
        text = split_pdf(finput)
    
    if not text:
        text = text22text(finput)
    else:
        print(df.loc[i, 'seqnum'])
    
    if text.isspace():
        text = image22text(finput, str(df.loc[i, 'seqnum']))
        ilist.append('1')
    else:
        ilist.append('0')
    tlist.append(text)
        
df['text'] = tlist
df['image'] = ilist
x2name = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', f"{'xlsx_#2_text'}.xlsx")
df.to_excel(x2name)

21680
21679
21665
21601
21505
21388
21387
21375
21335
21271
21249
21241
21234
21197
21196
21167
21148
21109
21100
21099
21054
21053
20923
20922
20920
20896
20860
20623
20622
20588
20539
20538
20537
20536
20535
20534
20533
20532
20531
20504
20503
20502
20491
20487
20485
20484
20483
20304
20303
20223
20211
20189
20167
20134
20128
20127
20019
20008
19980
19952
19929
19916
19900
19899
19897
19847
19805
19768
19765
19577
19572
19569
19562
19505
19479
19248
19247
19245
19217
19208
19207
19206
19205
19204
19203
19190
19185
19176
19166
19165
19164
19064
18992
18976
18967
18966
18959
18939
18865
18861
18814
18775
18745
18656
18543
18399
18019
18016
18010
17972
17947
17946
17945
17944
17943
17942
17941
17940
17939
17938
17937
17936
17935
17934
17933
17932
17931
17930
17928
17927
17892
17518
17454
17375
17205
17056
17050
17003
16695
16679
16546
16509
16503
16495
16484
16459
16335
16332
16282
16260
16255
16114
16064
16063
16045
16037
15966
15815
15809
15743
15712
15682
15670
15616
15615
15614
1560